In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD_avg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFNSD_avg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag, init_values=T_bot)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 2000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FNSD_avg%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD_avg%i.log' % k, mode='a')

tempf_prev = tempF.curr.copy()
fluxs_prev = fluxS.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    tempf_prev[:] = tempF.curr
    fluxs_prev[:] = fluxS.curr
    
    # mapping
    fluxF.curr[:] = fluxS.curr[s2f]
    
    solverF.put_heat_flux_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=fluxF.curr
    )
    
    while True:
        fluxF.backup()
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        tempS.curr[:] = 0.5*(tempF.curr[f2s]+tempf_prev[f2s])
        
        solverS.update_temperature_interface(interface_tag, tempS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        fluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        fluxF.curr[:] = fluxS.curr[s2f] 
        
        # update residual
        fluxF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(fluxF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to solid then restore
            if use_relax:
                under_relax.determine_omega(fluxF)
                under_relax.update_solution(fluxF)
            solverF.put_heat_flux_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(fluxF.curr+fluxs_prev[s2f])
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=14.
step=1, pc_iterations=15.
step=2, pc_iterations=14.
step=3, pc_iterations=15.
step=4, pc_iterations=14.
step=5, pc_iterations=15.
step=6, pc_iterations=15.
step=7, pc_iterations=16.
step=8, pc_iterations=14.
step=9, pc_iterations=15.
step=10, pc_iterations=15.
step=11, pc_iterations=15.
step=12, pc_iterations=15.
step=13, pc_iterations=16.
step=14, pc_iterations=14.
step=15, pc_iterations=15.
step=16, pc_iterations=14.
step=17, pc_iterations=15.
step=18, pc_iterations=14.
step=19, pc_iterations=15.
step=20, pc_iterations=15.
step=21, pc_iterations=15.
step=22, pc_iterations=15.
step=23, pc_iterations=15.
step=24, pc_iterations=15.
step=25, pc_iterations=15.
step=26, pc_iterations=15.
step=27, pc_iterations=15.
step=28, pc_iterations=15.
step=29, pc_iterations=15.
step=30, pc_iterations=15.
step=31, pc_iterations=15.
step=32, pc_iterations=14.
step=33, pc_iterations=15.
step=34, pc_iterations=14.
step=35, pc_iterations=15.
step=36, pc_iterations=14.
step=37, pc

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFNSD_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.9243504  0.93065952 0.93571373 0.93859167 0.94105206 0.94332584
 0.94534583 0.94718418 0.94896294 0.95055444 0.95204884 0.95344249
 0.95479962 0.95606655 0.95727979 0.95841583 0.95952576 0.96057629
 0.96157263 0.96254525 0.96346688 0.96436022 0.96521892 0.96603562
 0.96683334 0.96759644 0.96834014 0.96903979 0.96970867 0.97036383
 0.97099999 0.97160773 0.97219861 0.97277167 0.97332299 0.97385027
 0.97436845 0.97486586 0.97535381 0.97581604 0.97625985 0.97668783
 0.97710638 0.97751314 0.97790777 0.97828842 0.97866023 0.97901444
 0.97935629 0.97969507 0.980019   0.98032945 0.98062991 0.98092297
 0.98120387 0.98147867 0.98174021 0.98199437 0.98224181 0.98247583
 0.98269983 0.98292166 0.98313158 0.98333132 0.98352041 0.98370621
 0.98387872 0.98404406 0.98419676 0.98434422 0.98448793 0.98461745
 0.98473468 0.98484983 0.98495144 0.98504258 0.98512096 0.98519335
 0.98525083 0.98529887 0.98533117 0.98535297 0.98536441 0.98536552
 0.98534805 0.98531274 0.98526603 0.98519526 0.98510338 0.9849